## Корпус
1000 предложений со словом "человек" (0) и 1000 предложений со словом "пароход" (1) из основного корпуса НКРЯ. Заменены на Xx_xX

## Предобработка
Токенизация - word_tokenize из NLTK, лемматизация - pymorphy2, стоп-слова - из NLTK

In [45]:
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

orig_sents = []
sents = []
classes = []
morph = pymorphy2.MorphAnalyzer()
stopw = set(stopwords.words('russian'))
punct = string.punctuation + '`―«»'
with open('man-ship.txt','r',encoding='utf-8') as f:
    for line in f.readlines():
        if line.strip():
            cls = line.strip().split('\t')[0]
            sent = line.strip().split('\t')[1]
            orig_sents.append(sent)
            tokens = word_tokenize(sent)
            lemmas = [morph.parse(x)[0].normal_form for x in tokens if x.lower() not in stopw and x.strip(punct)]
            sents.append(' '.join(lemmas))
            classes.append(int(cls))

## Векторизация
Два типа - CountVectorizer и TfidfVectorizer

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer()

count_sents = count_vect.fit_transform(sents)
tfidf_sents = tfidf_vect.fit_transform(sents)

In [18]:
from sklearn.model_selection import train_test_split

X_count_train, X_count_test, y_count_train, y_count_test = train_test_split(count_sents, classes, test_size=0.3, random_state=42)
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(tfidf_sents, classes, test_size=0.3, random_state=42)

## Классификаторы
* Naive Bayes
* Logistic Regression
* SVM

In [19]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [20]:
bayes = MultinomialNB()
bayes.fit(X_count_train,y_count_train)
print('CountVectorizer + NaiveBayes:',f1_score(y_count_test,bayes.predict(X_count_test)))

bayes.fit(X_tfidf_train,y_tfidf_train)
print('TfIdfVectorizer + NaiveBayes:',f1_score(y_tfidf_test,bayes.predict(X_tfidf_test)))

CountVectorizer + NaiveBayes: 0.879338842975
TfIdfVectorizer + NaiveBayes: 0.891891891892


In [6]:
logistic = LogisticRegression(random_state=42)
logistic.fit(X_count_train,y_count_train)
print('CountVectorizer + LogisticRegression:',f1_score(y_count_test,logistic.predict(X_count_test)))

logistic.fit(X_tfidf_train,y_tfidf_train)
print('TfIdfVectorizer + LogisticRegression:',f1_score(y_tfidf_test,logistic.predict(X_tfidf_test)))

CountVectorizer + LogisticRegression: 0.863945578231
TfIdfVectorizer + LogisticRegression: 0.867671691792


In [7]:
svm = LinearSVC(random_state=42)
svm.fit(X_count_train,y_count_train)
print('CountVectorizer + SVM:',f1_score(y_count_test,svm.predict(X_count_test)))

svm.fit(X_tfidf_train,y_tfidf_train)
print('TfIdfVectorizer + SVM:',f1_score(y_tfidf_test,svm.predict(X_tfidf_test)))

CountVectorizer + SVM: 0.849829351536
TfIdfVectorizer + SVM: 0.877721943049


|       | NaiveBayes     | LogisticRegression | LinearSVC      |
|-------|----------------|--------------------|----------------|
| Count | 0.879338842975 | 0.863945578231     | 0.849829351536 |
| TfIdf | 0.891891891892 | 0.867671691792     | 0.877721943049 |

Лучше всего работает наивный байесовский классификатор с tf-idf векторизатором.

## Уменьшение размерности
Минимальная частота слова - 5, максимальная - 500. Тестируем на логистической регрессии

In [8]:
small_count_vect = CountVectorizer(min_df=5,max_df=500)
small_tfidf_vect = TfidfVectorizer(min_df=5,max_df=500)

count_sents = small_count_vect.fit_transform(sents)
tfidf_sents = small_tfidf_vect.fit_transform(sents)

X_count_train, X_count_test, y_count_train, y_count_test = train_test_split(count_sents, classes, test_size=0.3, random_state=42)
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(tfidf_sents, classes, test_size=0.3, random_state=42)

logistic = LogisticRegression(random_state=42)
logistic.fit(X_count_train,y_count_train)
print('CountVectorizer + LogisticRegression:',f1_score(y_count_test,logistic.predict(X_count_test)))

logistic.fit(X_tfidf_train,y_tfidf_train)
print('TfIdfVectorizer + LogisticRegression:',f1_score(y_tfidf_test,logistic.predict(X_tfidf_test)))

CountVectorizer + LogisticRegression: 0.843333333333
TfIdfVectorizer + LogisticRegression: 0.852404643449


Работает хуже.

## Ошибки  (NaiveBayes + TfIdf)

In [58]:
import random

cls_pred = bayes.predict(tfidf_sents)
errors = random.sample([(i,classes[i],x) for i,x in enumerate(cls_pred) if x != classes[i]],5)
print('true','pred','sent')
for e in errors:
    print(e[1],e[2],orig_sents[e[0]])

true pred sent
1 0  Хотя бы ради того, чтобы прикоснуться к советской жизни, на самый короткий срок, быстрым Xx_xX, без всяких задержек в Европе, получить ваши указания, урегулировать ряд практических дел…».
0 1  А он, деликатный Xx_xX, попросил, чтобы охрана поехала сзади.
1 0  Процесс производства, например, импортного масла «Анкор» выглядит так: 25-килограммовые блоки масла Xx_xX приплывают из Новой Зеландии, размягчаются и фасуются в стандартные пачки.
1 0  ― Как это так, ведь Xx_xX больше не приходил.
0 1  Xx_xX, передавший бумаги, рассказал, что служебных записок от Дурманова было несколько ― написанных в разное время, но на одну тему.


Ошибки (кроме четвертой, пожалуй) странные, не очень понятно, какие слова их вызывают.

## Примеры работы (NaiveBayes + TfIdf)

In [22]:
new_sents_man = [
    'И сама жизнь Xx_xX ― это необратимый процесс.',
    'Также ясно вижу и масштаб проблем, которые были бы связаны с любым Xx_xX, который имел шансы стать президентом России вместо Путина.',
    'Раньше считалось, что в России средний Xx_xX может прожить на 1137 руб. в месяц.',
    'Ты ведь взрослый Xx_xX и этого парнечка знаешь…',
    'В колхозной конторе окна не светили, но на гул машин вышли два Xx_xX: взрослый и мальчишка.'
]

new_sents_ship = [
    'Как только Xx_xX подваливал к пристани, они начинали оплакивать умершего высокими, томительными голосами.',
    'Прилепившись присосками к отвесной скале, над затонувшим Xx_xX висел плоский диск.',
    'Несколько взмахов весел донесли нас к борту югославского Xx_xX.',
    'Xx_xX опоздал на шесть часов и только утром ушел из Казани.',
    ' Нос отделился от кормы, и обе части Xx_xX, снятые с камней экспедицией Эпрона, стояли рядом, покачиваясь на якорях…'
]

In [23]:
print('true','pred','sent')
print('man')
for sent in new_sents_man:
    tokens = word_tokenize(sent)
    lemmas = [morph.parse(x)[0].normal_form for x in tokens if x.lower() not in stopw and x.strip(punct)]
    vect = tfidf_vect.transform([' '.join(lemmas)])
    pred = bayes.predict(vect)
    print(0,pred[0],sent)
    
print('\nship')
for sent in new_sents_ship:
    tokens = word_tokenize(sent)
    lemmas = [morph.parse(x)[0].normal_form for x in tokens if x.lower() not in stopw and x.strip(punct)]
    vect = tfidf_vect.transform([' '.join(lemmas)])
    pred = bayes.predict(vect)
    print(1,pred[0],sent)

true pred sent
man
0 0 И сама жизнь Xx_xX ― это необратимый процесс.
0 0 Также ясно вижу и масштаб проблем, которые были бы связаны с любым Xx_xX, который имел шансы стать президентом России вместо Путина.
0 0 Раньше считалось, что в России средний Xx_xX может прожить на 1137 руб. в месяц.
0 0 Ты ведь взрослый Xx_xX и этого парнечка знаешь…
0 1 В колхозной конторе окна не светили, но на гул машин вышли два Xx_xX: взрослый и мальчишка.

ship
1 1 Как только Xx_xX подваливал к пристани, они начинали оплакивать умершего высокими, томительными голосами.
1 1 Прилепившись присосками к отвесной скале, над затонувшим Xx_xX висел плоский диск.
1 1 Несколько взмахов весел донесли нас к борту югославского Xx_xX.
1 1 Xx_xX опоздал на шесть часов и только утром ушел из Казани.
1 1  Нос отделился от кормы, и обе части Xx_xX, снятые с камней экспедицией Эпрона, стояли рядом, покачиваясь на якорях…


Одна ошибка в примерах на "человек", вероятно, из-за слова "гул" и, может быть, "машин".